In [1]:
import heapq

# https://atcoder.jp/contests/practice2/submissions/16805680
class mcf_graph_int_cost:
    """
    頂点数、及び、costの総和が、4294967295 (== (1 << 32) - 1) を超えない前提下での高速な実装。
    後者は超えても一応動く。
    """
 
    def __init__(self, n):
        self.n = n
        self.pos = []
        self.g = [[] for _ in range(n)]
 
 
    def add_edge(self, from_, to, cap, cost):
        # assert 0 <= from_ < self.n
        # assert 0 <= to < self.n
        m = len(self.pos)
        self.pos.append((from_, len(self.g[from_])))
        self.g[from_].append(self.__class__._edge(to, len(self.g[to]), cap, cost))
        self.g[to].append(self.__class__._edge(from_, len(self.g[from_]) - 1, 0, -cost))
        return m
 
 
    class edge:
        def __init__(self, from_, to, cap, flow, cost):
            self.from_ = from_
            self.to = to
            self.cap = cap
            self.flow = flow
            self.cost = cost
 
 
    def get_edge(self, i):
        _e = self.g[self.pos[i][0]][self.pos[i][1]]
        _re = self.g[_e.to][_e.rev]
        return self.__class__.edge(self.pos[i][0], _e.to, _e.cap + _re.cap, _re.cap, _e.cost)
 
 
    def edges(self):
        ret = []
        for i in range(len(self.pos)):
            _e = self.g[self.pos[i][0]][self.pos[i][1]]
            _re = self.g[_e.to][_e.rev]
            ret.append(self.__class__.edge(self.pos[i][0], _e.to, _e.cap + _re.cap, _re.cap, _e.cost))
        return ret
 
 
    def _dual_ref(self):
        self.dist = [4294967295] * self.n
        self.pv = [-1] * self.n
        self.pe = [-1] * self.n
        self.vis = [False] * self.n
 
        que = [s]
        self.dist[s] = 0
        while que:
            v = heapq.heappop(que) & 4294967295
            if self.vis[v]:
                continue
            self.vis[v] = True
            if v == t:
                break
            for i in range(len(self.g[v])):
                e = self.g[v][i]
                if self.vis[e.to] or e.cap == 0:
                    continue
                cost = e.cost - self.dual[e.to] + self.dual[v]
                if self.dist[e.to] > self.dist[v] + cost:
                    self.dist[e.to] = self.dist[v] + cost
                    self.pv[e.to] = v
                    self.pe[e.to] = i
                    assert ((self.dist[e.to] << 32) + e.to) < 2 ** 64 # 興味本位
                    heapq.heappush(que, ((self.dist[e.to] << 32) + e.to))
        if not self.vis[t]:
            return False
 
        for v in range(self.n):
            if not self.vis[v]:
                continue
            self.dual[v] -= self.dist[t] - self.dist[v]
        
        return True
 
 
    def slope(self, s, t, flow_limit=4294967295):
        # assert 0 <= s < self.n
        # assert 0 <= t < self.n
        # assert s != t
        
        self.dual = [0] * self.n
        self.dist = [4294967295] * self.n
        self.pv = [-1] * self.n
        self.pe = [-1] * self.n
        self.vis = [False] * self.n
        
        flow = 0
        cost = 0
        prev_cost = -1
        result = [(flow, cost)]
        while flow < flow_limit:
            if not self._dual_ref():
                break
            c = flow_limit - flow
            v = t
            while v != s:
                c = min(c, self.g[self.pv[v]][self.pe[v]].cap)
                v = self.pv[v]
            v = t
            while v != s:
                e = self.g[self.pv[v]][self.pe[v]]
                e.cap -= c
                self.g[v][e.rev].cap += c
                v = self.pv[v]
            d = -self.dual[s]
            flow += c
            cost += c * d
            if prev_cost == d:
                result.pop()
            result.append((flow, cost))
            prev_cost = cost
        return result
 
 
    def flow(self, s, t, flow_limit=4294967295):
        return self.slope(s, t, flow_limit)[-1]
 
    
    class _edge:
        def __init__(self, to, rev, cap, cost):
            self.to = to
            self.rev = rev
            self.cap = cap
            self.cost = cost
 
 
# BIG = 10 ** 9
 
# import sys
# input = sys.stdin.readline
 
# N, K = map(int, input().split())
 
# g = mcf_graph_int_cost(2 * N + 2)
# s = 2 * N
# t = 2 * N + 1
 
# g.add_edge(s, t, N * K, BIG)
# for i in range(N):
#     g.add_edge(s, i, K, 0)
#     g.add_edge(N + i, t, K, 0)
 
# for i in range(N):
#     As = map(int, input().split())
#     for j, A in enumerate(As):
#         g.add_edge(i, N + j, 1, BIG - A)
 
# result = g.flow(s, t, N * K)
 
# print(N * K * BIG - result[1])
 
# grid = [['.' for _ in range(N)] for _ in range(N)]
# edges = g.edges()
# for e in edges:
#     if e.from_ == s or e.to == t or e.flow == 0:
#         continue
#     grid[e.from_][e.to - N] = 'X'
 
# for row in grid:
#     print(''.join(row))


class Solution:
    def connectTwoGroups(self, cost: List[List[int]]) -> int:
        left = len(cost)
        right = len(cost[0])
        
        g = mcf_graph_int_cost(left+right+2)
        for i,lst in enumerate(cost):
            g.add_edge(i, left+right, 100, 0)
            for j,c in enumerate(lst):
                if j == 0:
                    g.add_edge(j+left, left+right+1, 1, 0)
                g.add_edge(i, j+left, 1, c)
        
        g.add_edge(left+right, left+right+1, 10**9, 10**9)
        
        result = g.flow(left+right, left+right+1)
        print(result)
        return result
    
        # choose minimum in every row and column
        # for i,x in cost:
            
            
    
    
                
        

[[2, 3, 4], [0, 3, 5], [0, 1, 10]]

In [ ]:
from heapq import heappush, heappop
# https://atcoder.jp/contests/practice2/submissions/16741434
class MinCostFlow:
    INF = 10**18
 
    def __init__(self, N):
        self.N = N
        self.G = [[] for i in range(N)]
 
    def add_edge(self, fr, to, cap, cost):
        forward = [to, cap, cost, None]
        backward = forward[3] = [fr, 0, -cost, forward]
        self.G[fr].append(forward)
        self.G[to].append(backward)
 
    def flow(self, s, t, f):
        N = self.N; G = self.G
        INF = MinCostFlow.INF
 
        res = 0
        H = [0]*N
        prv_v = [0]*N
        prv_e = [None]*N
 
        d0 = [INF]*N
        dist = [INF]*N
 
        while f:
            dist[:] = d0
            dist[s] = 0
            que = [(0, s)]
 
            while que:
                c, v = heappop(que)
                if dist[v] < c:
                    continue
                r0 = dist[v] + H[v]
                for e in G[v]:
                    w, cap, cost, _ = e
                    if cap > 0 and r0 + cost - H[w] < dist[w]:
                        dist[w] = r = r0 + cost - H[w]
                        prv_v[w] = v; prv_e[w] = e
                        heappush(que, (r, w))
            if dist[t] == INF:
                return None
 
            for i in range(N):
                H[i] += dist[i]
 
            d = f; v = t
            while v != s:
                d = min(d, prv_e[v][1])
                v = prv_v[v]
            f -= d
            res += d * H[t]
            v = t
            while v != s:
                e = prv_e[v]
                e[1] -= d
                e[3][1] += d
                v = prv_v[v]
        return res
 
# N,K = map(int,input().split())
# A = [list(map(int,input().split())) for i in range(N)]
 
# G = MinCostFlow(2*N+2)
# G.add_edge(0,2*N+1,10**18,10**9)
 
# for i in range(N):
#     G.add_edge(0,i+1,K,0)
#     for j in range(N):
#         G.add_edge(i+1,N+1+j,1,10**9-A[i][j])
 
# for j in range(N):
#     G.add_edge(N+1+j,2*N+1,K,0)
 
# f = G.flow(0,2*N+1,N**2)
# print(N*N*10**9-f)

class Solution:
    def connectTwoGroups(self, cost: List[List[int]]) -> int:
        left = len(cost)
        right = len(cost[0])
        
        g = MinCostFlow(left+right + 2)
        for i,lst in enumerate(cost):
            g.add_edge(left+right, i, 1, 0)
            for j,c in enumerate(lst):
                if j == 0:
                    g.add_edge(j+left, left+right + 1, 100, 0)
                g.add_edge(i, j+left, 1, c)
                
        g.add_edge(left+right, left+right+1, 10**9, 10**9)
        
        result = g.flow(left+right, left+right+1, 10**9)
        print(result)
        print((10**9-1)**2-result)
        return result
    
        # choose minimum in every row and column
        # for i,x in cost:
            
            
    
    
                
        